In [19]:
from prophet import Prophet
import pandas as pd

In [20]:
import codecs
with codecs.open("./data/nikkei_stock_average_monthly_jp.csv", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")
df.head()

,データ日付,終値,始値,高値,安値
0,2000/01/01,19539.70,19002.86,19539.70,18168.27
1,2000/02/01,19959.52,19423.38,20007.77,19367.83
2,2000/03/01,20337.32,20081.67,20706.65,19078.60
3,2000/04/01,17973.70,20726.99,20833.21,17973.70
4,2000/05/01,16332.45,18403.08,18439.36,16008.14


In [21]:
df = df.rename(columns={"データ日付":"date", "終値":"closing_price", "始値":"open_price", "高値":"higher_price", "安値":"lower_price"})
df = df[:-1]
print(len(df))
df.head()

267


,date,closing_price,open_price,higher_price,lower_price
0,2000/01/01,19539.70,19002.86,19539.70,18168.27
1,2000/02/01,19959.52,19423.38,20007.77,19367.83
2,2000/03/01,20337.32,20081.67,20706.65,19078.60
3,2000/04/01,17973.70,20726.99,20833.21,17973.70
4,2000/05/01,16332.45,18403.08,18439.36,16008.14


In [22]:
#日付の処理
df["date"] = pd.to_datetime(df["date"])
df.head()

,date,closing_price,open_price,higher_price,lower_price
0,2000-01-01,19539.70,19002.86,19539.70,18168.27
1,2000-02-01,19959.52,19423.38,20007.77,19367.83
2,2000-03-01,20337.32,20081.67,20706.65,19078.60
3,2000-04-01,17973.70,20726.99,20833.21,17973.70
4,2000-05-01,16332.45,18403.08,18439.36,16008.14


In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
  File "/var/folders/b2/sz8856894cv9zhqvldw4j7780000gn/T/ipykernel_23361/2879353197.py", line 1, in <cell line: 1>
    import matplotlib.pyplot as plt
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/matplotlib/pyplot.py", line 2230, in <module>
    switch_backend(rcParams["backend"])
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/matplotlib/pyplot.py", line 267, in switch_backend
    class backend_mod(matplotlib.backend_bases._Backend):
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/matplotlib/pyplot.py", line 268, in backend_mod
    locals().update(vars(importlib.import_module(backend_name)))
  File "/Users/harusaku/anaconda3/lib/python3.9/

In [24]:
t = df.date
for i in range(1,4):
    y = df.iloc[:, i]
    plt.plot(t,y, label=df.columns[i])
plt.legend()
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
  File "/var/folders/b2/sz8856894cv9zhqvldw4j7780000gn/T/ipykernel_23361/2058804726.py", line 4, in <cell line: 2>
    plt.plot(t,y, label=df.columns[i])
NameError: name 'plt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/pygments/styles/__init__.py", line 78, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/harusaku/Desktop/HaitLab/Stock_price_prediction_app/.env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
  File "/Users/harus

In [ ]:
#予測
m = Prophet(seasonality_mode="multiplicative")
df_closing = df[["date", "closing_price"]].rename(columns={"date":"ds", "closing_price":"y"})
m.fit(df_closing)

In [ ]:
future = m.make_future_dataframe(periods=120, freq="MS")
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()

In [ ]:
fig1 = m.plot(forecast)
print(type(fig1))

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
import pandas as pd
import pandas_datareader.data as web

In [ ]:
df = web.DataReader("9020.T", data_source="yahoo", start="2000-01-01")
df.head()

In [ ]:
df.shape
df.reset_index(inplace=True)
df.columns

In [ ]:
df_predict = df[["Date", "Adj Close"]].rename(columns={"Date":"ds", "Adj Close":"y"})
df_predict.head()

In [ ]:
m = Prophet()
m.fit(df_predict)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(10)

In [ ]:
fig1 = m.plot(forecast)
type(fig1)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
df = pd.read_excel("../managing_app/templates/data_j.xlsx")
df = df[["コード","銘柄名","市場・商品区分", "33業種コード", "33業種区分"]]
df.head()

In [ ]:
special = df[df["33業種コード"] == "-"]
special

In [ ]:
igai33 = list(set(special["市場・商品区分"].tolist()))
igai33

In [ ]:
data = {}
for content in igai33:
    kind = special[special["市場・商品区分"]==content]
    data[content] = kind["銘柄名"].tolist()
    print(len(data[content]))
kind["銘柄名"]

In [ ]:
data

In [ ]:
import numpy as np
df2 = df.replace({"-": np.nan})

In [ ]:
df_33 = df[(df["33業種コード"] != "-") & (df["33業種区分"] != "-")] 
df_33

In [ ]:
gyousyu33 = list(set(df_33["33業種区分"].tolist()))
gyousyu33

In [ ]:
for content in gyousyu33:
    kind = df_33[df_33["33業種区分"] == content]
    data[content] = kind["銘柄名"].tolist()
data

In [ ]:
print(data.keys())

In [ ]:
# データの受け取り先は改訂予定
df = pd.read_excel("../managing_app/templates/data_j.xlsx")
df = df[["コード","銘柄名","市場・商品区分", "33業種コード", "33業種区分"]]

#33業種以外の物を抽出
special = df[df["33業種コード"] == "-"]

#33業種以外の銘柄名をリスト化
igai33 = list(set(special["市場・商品区分"].tolist()))

#業種ごとに銘柄を辞書でまとめる1
data = {}
for content in igai33:
    kind = special[special["33業種区分"] == content]
    data[content] = kind["銘柄名"].tolist()

#33業種を抽出
df_33 = df[(df["33業種コード"] != "-") & (df["33業種区分"] != "-")] 

# 33業種をリスト化

gyousyu33 = list(set(df_33["33業種区分"].tolist()))

#業種ごとに銘柄を辞書でまとめる2
for content in gyousyu33:
    kind = df_33[df_33["33業種区分"] == content]
    data[content] = kind["銘柄名"].tolist()

gyousyu33.extend(igai33)

In [ ]:
data